<a href="https://colab.research.google.com/github/dragontea72/ML-ZoomCamp-Assignment/blob/main/homework_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset
In this homework, we will use the California Housing Prices from Kaggle.

Here's a wget-able link:

wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

The goal of this homework is to create a regression model for predicting housing prices (column 'median_house_value').

In [105]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [106]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

--2022-09-20 09:51:23--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv.3’

housing.csv.3       100%[===================>]   1.36M  --.-KB/s    in 0.06s   

2022-09-20 09:51:23 (23.9 MB/s) - ‘housing.csv.3’ saved [1423529/1423529]



In [107]:
df_dataset = pd.read_csv("housing.csv")
df_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [108]:
df_dataset.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object

In [109]:
df_dataset['median_house_value']

0        452600.0
1        358500.0
2        352100.0
3        341300.0
4        342200.0
           ...   
20635     78100.0
20636     77100.0
20637     92300.0
20638     84700.0
20639     89400.0
Name: median_house_value, Length: 20640, dtype: float64

In [110]:
df = df_dataset.drop('ocean_proximity', axis =1)
df.head(2)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0


## Question 1

In [45]:
df.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

`Total missing values are 207.`

## Question 2

In [46]:
print("The median for variable Population", df['population'].median())

The median for variable Population 1166.0


## Split the data
- Shuffle the initial dataset, use seed 42.
- Split your data in train/val/test sets, with 60%/20%/20% distribution.
- Make sure that the target value ('median_house_value') is not in your dataframe.
- Apply the log transformation to the median_house_value variable using the np.log1p() function.

## Question 3
- We need to deal with missing values for the column from Q1.

- We have two options: fill it with 0 or with the mean of this variable.
- Try both options. For each, train a linear regression model without regularization using the code from the lessons.
- For computing the mean, use the training only!
- Use the validation dataset to evaluate the models and compare the RMSE of each option.
- Round the RMSE scores to 2 decimal digits using round(score, 2)
Which option gives better RMSE?

### Train, Test, & Validation set

In [137]:
n = len(df)
n_val = int(n*0.2)
n_test = int(n*0.2)
n_train = n - n_val - n_test

print("n: ", n)
print("n_val: ", n_val)
print("n_test: ", n_test)
print("n_train: ", n_train)

n:  20640
n_val:  4128
n_test:  4128
n_train:  12384


In [138]:
n, n_val + n_test + n_train

(20640, 20640)

In [139]:
idx = np.arange(n)
idx

array([    0,     1,     2, ..., 20637, 20638, 20639])

In [140]:
#np.random.seed(42)
np.random.seed(9) # for question 9
np.random.shuffle(idx)

In [141]:
df_train = df.iloc[idx[:n_train]]
df_val = df.iloc[idx[n_train : n_train + n_val]]
df_test = df.iloc[idx[n_train + n_val : ]]

In [142]:
df_train.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
20558,-121.80,38.67,10.0,2086.0,380.0,1073.0,378.0,4.5526,154400.0
11430,-117.97,33.66,22.0,3914.0,600.0,1871.0,607.0,5.8541,281500.0
2656,-124.25,40.28,32.0,1430.0,419.0,434.0,187.0,1.9417,76100.0


In [143]:
df_test.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
168,-122.24,37.79,27.0,1632.0,492.0,1171.0,429.0,2.3173,125000.0
17299,-119.51,34.40,15.0,1112.0,256.0,411.0,245.0,2.0625,314300.0
10347,-117.66,33.61,21.0,1932.0,266.0,860.0,286.0,7.1497,274000.0


In [144]:
df_val.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
13339,-117.66,34.03,14.0,2137.0,345.0,1151.0,352.0,5.7530,185500.0
2590,-124.09,40.88,31.0,1982.0,495.0,1052.0,467.0,1.5326,74100.0
18494,-121.57,37.02,17.0,2889.0,624.0,2681.0,608.0,2.9417,178000.0


In [145]:
len(df_train), len(df_val), len(df_test)

(12384, 4128, 4128)

In [146]:
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [147]:
y_train = np.log1p(df_train.median_house_value.values)
y_val = np.log1p(df_val.median_house_value.values)
y_test = np.log1p(df_test.median_house_value.values)

In [148]:
del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

In [149]:
df_train.head(2)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,-121.80,38.67,10.0,2086.0,380.0,1073.0,378.0,4.5526
1,-117.97,33.66,22.0,3914.0,600.0,1871.0,607.0,5.8541


In [150]:
y_train

array([11.94730839, 12.54789128, 11.23981668, ..., 12.11888489,
       12.65937826, 12.20607765])

### Dealing with missing values with 0 and mean method

In [92]:
df_train.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
dtype: object

In [93]:
df_train.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
dtype: int64

In [125]:
base = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']
base

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income']

In [95]:
df_train[base].isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
dtype: int64

In [65]:
X_train = df_train[base].fillna(0).values

In [97]:
total_bedrooms_mean = df_train['total_bedrooms'].mean()
total_bedrooms_mean

533.4803317730147

In [98]:
X_train_mean = df_train[base].fillna(total_bedrooms_mean).values

In [37]:
#df_train['total_bedrooms'] = df_train['total_bedrooms'].fillna(0)

In [38]:
#df_train.isnull().sum()

### Train Linear Regression Model

In [35]:
def train_linear_regression(X, y):
  ones = np.ones(X.shape[0])
  X = np.column_stack([ones, X])

  XTX = X.T.dot(X)
  XTX_inv = np.linalg.inv(XTX)
  w_full = XTX_inv.dot(X.T).dot(y)

  return w_full[0], w_full[1:]

In [66]:
w0, w = train_linear_regression(X_train, y_train)
w0, w

(-11.686975242036546,
 array([-2.76255120e-01, -2.82087184e-01,  2.91329621e-03, -8.64531514e-06,
         1.50811922e-04, -2.08205659e-04,  5.69546573e-04,  1.71308139e-01]))

In [67]:
y_pred = w0 + X_train.dot(w)
y_pred[0:10]

array([11.28055213, 11.90427911, 12.46507934, 12.37306561, 12.40607194,
       11.9746456 , 12.30699616, 12.038972  , 12.41974316, 12.83046065])

In [99]:
w0, w = train_linear_regression(X_train_mean, y_train)
w0, w

(-11.759590872680436,
 array([-2.76809637e-01, -2.82182103e-01,  2.93033801e-03, -1.48543785e-05,
         2.48054046e-04, -2.03763456e-04,  4.87004092e-04,  1.73086593e-01]))

In [100]:
y_pred = w0 + X_train_mean.dot(w)
y_pred[0:10]

array([11.37346084, 11.97238956, 12.49872828, 12.45362567, 12.49192508,
       12.01852362, 12.41479165, 12.14215554, 12.49444534, 12.92941654])

### Root Mean Squared Error

In [34]:
def rmse(y, y_pred):
  se = (y - y_pred)**2
  mse = se.mean()
  return np.sqrt(mse)


In [153]:
def prepare_X(df):
  df_num = df[base]
  df_num = df_num.fillna(0)
  X = df_num.values
  return X

In [71]:
X_val = prepare_X(df_val)
y_pred = w0 + X_val.dot(w)

RMSE = round(rmse(y_val, y_pred), 2)
print("RMSE with 0 : ", RMSE)

RMSE with 0 :  0.33


In [101]:
total_bedrooms_mean_val = df_val['total_bedrooms'].mean()
total_bedrooms_mean_val

543.6441375968992

In [103]:
X_val_mean = df_val[base].fillna(total_bedrooms_mean_val).values

In [104]:
y_pred_mean = w0 + X_val_mean.dot(w)

RMSE_mean = round(rmse(y_val, y_pred_mean), 2)
print("RMSE with mean : ", RMSE)

RMSE with mean :  0.33


## Question 4
- Now let's train a regularized linear regression.
- For this question, fill the NAs with 0.
- Try different values of r from this list: [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10].
- Use RMSE to evaluate the model on the validation dataset.
- Round the RMSE scores to 2 decimal digits.
- Which r gives the best RMSE?

In [128]:
def train_linear_regression_reg(X, y, r=0.001):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX = XTX + r * np.eye(XTX.shape[0]) # Regularization

    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

In [130]:
for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
    X_train = prepare_X(df_train)
    w0, w = train_linear_regression_reg(X_train, y_train, r=r)

    X_val = prepare_X(df_val)
    y_pred = w0 + X_val.dot(w)
    score = round(rmse(y_val, y_pred), 2)
    print(r, w0, score)

0 -11.686975242036546 0.33
1e-06 -11.686959175728921 0.33
0.0001 -11.685368865034667 0.33
0.001 -11.670931317799797 0.33
0.01 -11.528493585632091 0.33
0.1 -10.274500282195673 0.33
1 -4.920480897742212 0.33
5 -1.482095745621172 0.34
10 -0.7899311831779245 0.34


### Question 5
- We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
- Try different seed values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9].
- For each seed, do the train/validation/test split with 60%/20%/20% distribution.
- Fill the missing values with 0 and train a model without regularization.
- For each seed, evaluate the model on the validation dataset and collect the RMSE scores.
- What's the standard deviation of all the scores? To compute the standard deviation, use np.std.
- Round the result to 3 decimal digits (round(std, 3))

## Question 6
- Split the dataset like previously, use seed 9.
- Combine train and validation datasets.
- Fill the missing values with 0 and train a model with r=0.001.
- What's the RMSE on the test dataset?

In [151]:
df_full_train = pd.concat([df_train, df_val])
df_full_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,-121.80,38.67,10.0,2086.0,380.0,1073.0,378.0,4.5526
1,-117.97,33.66,22.0,3914.0,600.0,1871.0,607.0,5.8541
2,-124.25,40.28,32.0,1430.0,419.0,434.0,187.0,1.9417
3,-117.11,32.69,37.0,2395.0,627.0,2489.0,599.0,1.5933
4,-118.37,34.21,36.0,1392.0,326.0,1181.0,303.0,3.1563


In [152]:
df_full_train = df_full_train.reset_index(drop = True)
df_full_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,-121.80,38.67,10.0,2086.0,380.0,1073.0,378.0,4.5526
1,-117.97,33.66,22.0,3914.0,600.0,1871.0,607.0,5.8541
2,-124.25,40.28,32.0,1430.0,419.0,434.0,187.0,1.9417
3,-117.11,32.69,37.0,2395.0,627.0,2489.0,599.0,1.5933
4,-118.37,34.21,36.0,1392.0,326.0,1181.0,303.0,3.1563
...,...,...,...,...,...,...,...,...
16507,-122.30,38.31,34.0,1797.0,395.0,1162.0,407.0,3.4550
16508,-118.54,34.27,28.0,2309.0,300.0,931.0,302.0,6.7415
16509,-118.13,33.93,34.0,2122.0,517.0,1578.0,488.0,3.1496
16510,-118.37,34.11,42.0,5518.0,979.0,1863.0,957.0,8.5842


In [154]:
X_full_train = prepare_X(df_full_train)
X_full_train

array([[-121.8   ,   38.67  ,   10.    , ..., 1073.    ,  378.    ,
           4.5526],
       [-117.97  ,   33.66  ,   22.    , ..., 1871.    ,  607.    ,
           5.8541],
       [-124.25  ,   40.28  ,   32.    , ...,  434.    ,  187.    ,
           1.9417],
       ...,
       [-118.13  ,   33.93  ,   34.    , ..., 1578.    ,  488.    ,
           3.1496],
       [-118.37  ,   34.11  ,   42.    , ..., 1863.    ,  957.    ,
           8.5842],
       [-118.37  ,   34.08  ,   52.    , ...,  543.    ,  213.    ,
           6.0338]])

In [158]:
y_full_train = np.concatenate([y_train, y_val])

In [159]:
def train_linear_regression_reg(X, y, r=0.001):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX = XTX + r * np.eye(XTX.shape[0]) # Regularization

    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

In [161]:
w0, w = train_linear_regression_reg(X_full_train, y_full_train)

X_test = prepare_X(df_test)
y_pred = w0 + X_test.dot(w)
score = round(rmse(y_test, y_pred), 2)
print(score)

0.35
